<a href="https://colab.research.google.com/github/TakaNori999/ML_Class/blob/main/CNN_Grad_CAM%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title データの作成
# モジュール
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical

# データのダウンロード
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype("float32") / 255.0)[..., None]  # (N,28,28,1)
X_test  = (X_test.astype("float32")  / 255.0)[..., None]
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [19]:
#@title 中間層の構成
# 中間層の構成
UNITS_COV1 = 32 #@param {type:"slider", min:32, max:512, step:32}
UNITS_COV2 = 0 #@param {type:"slider", min:0, max:512, step:32}
UNITS_COV3 = 0 #@param {type:"slider", min:0, max:512, step:32}
UNITS_COV4 = 0 #@param {type:"slider", min:0, max:512, step:32}
UNITS_COV5 = 0 #@param {type:"slider", min:0, max:512, step:32}

In [ ]:
#@title 学習モデルのトレーニング
EPOCHS = 4 #@param {type:"slider", min:1, max:20, step:1}

# CNN（空間サイズを維持：pooling無し, stride=1, same）
inputs = tf.keras.Input(shape=(28,28,1), name="inp")
x = tf.keras.layers.Conv2D(UNITS_COV1, (3,3), padding="same", activation="relu", name="conv1")(inputs)
if UNITS_COV2 > 0:
  x = tf.keras.layers.Conv2D(UNITS_COV2, (3,3), padding="same", activation="relu", name="conv2")(x)
if UNITS_COV3 > 0:
  x = tf.keras.layers.Conv2D(UNITS_COV3, (3,3), padding="same", activation="relu", name="conv3")(x)
if UNITS_COV4 > 0:
  x = tf.keras.layers.Conv2D(UNITS_COV4, (3,3), padding="same", activation="relu", name="conv4")(x)
if UNITS_COV5 > 0:
  x = tf.keras.layers.Conv2D(UNITS_COV5, (3,3), padding="same", activation="relu", name="conv5")(x)
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(64, activation="relu", name="fc")(x)
logits = tf.keras.layers.Dense(10, name="logits")(x)
outputs = tf.keras.layers.Activation("softmax", name="softmax")(logits)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, verbose=1, validation_data=(X_test, y_test))
test_acc = model.evaluate(X_test, y_test, verbose=0)[1]
print("Test ACC:", test_acc)


# 学習履歴の表示
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# --- Accuracy ---
ax[0].plot(epochs, acc, marker='o', label='Train Acc')
ax[0].plot(epochs, val_acc, marker='s', label='Val Acc')
ax[0].set_title('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_ylim(0, 1.0)
ax[0].grid(True)
ax[0].legend()

# --- Loss ---
ax[1].plot(epochs, loss, marker='o', label='Train Loss')
ax[1].plot(epochs, val_loss, marker='s', label='Val Loss')
ax[1].set_title('Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_ylim(0, None)
ax[1].grid(True)
ax[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
#@title テスト画像の表示
# Grad=CAM++
def gradcampp_for_layer(model, img4d, layer_name, eps=1e-8):
    target = model.get_layer(layer_name)
    grad_model = tf.keras.Model(
        inputs=model.inputs,
        outputs=[target.output, model.get_layer('logits').output]
    )

    with tf.GradientTape() as tape:
        fmap, logits = grad_model(img4d, training=False)          # fmap: (1,h,w,k)
        class_idx = tf.argmax(logits[0])
        score = logits[:, class_idx]                               # y^c

    grads = tape.gradient(score, fmap)                             # (1,h,w,k)
    fmap  = fmap[0]                                                # (h,w,k)
    grads = grads[0]                                               # (h,w,k)

    grads2 = tf.square(grads)                                      # g^2
    grads3 = grads2 * grads                                        # g^3
    alpha_num   = grads2
    alpha_denom = 2.0 * grads2 + fmap * grads3 + eps               # 2g^2 + A*g^3 + ε
    alpha = alpha_num / alpha_denom                                # (h,w,k)

    positive_grads = tf.nn.relu(grads)                             # ReLU(g)
    weights = tf.reduce_sum(alpha * positive_grads, axis=[0,1])    # w_k = Σ_ij α_ij ReLU(g_ij)

    cam = tf.reduce_sum(fmap * weights, axis=-1)                   # Σ_k w_k A^k
    cam = tf.nn.relu(cam).numpy()

    if cam.max() > 0:
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    H, W = img4d.shape[1], img4d.shape[2]
    cam = tf.image.resize(cam[..., None], (H, W)).numpy().squeeze()
    return cam, int(class_idx)

# 画像の白黒反転
USE_INVERT_GRAY = True
def show_overlay(img2d, heat2d, title="", alpha=0.5, cmap="jet"):
    if USE_INVERT_GRAY:
        plt.imshow(img2d, cmap="gray_r")
    else:
        plt.imshow(img2d, cmap="gray")
    plt.imshow(heat2d, cmap=cmap, alpha=alpha)
    plt.axis("off"); plt.title(title)

# テスト画像の表示
N = 40
np.random.seed(0)
gallery_idx = np.random.choice(len(X_test), size=N, replace=False)
gallery_map = {i: int(idx) for i, idx in enumerate(gallery_idx)}
rows, cols = 5, 8
plt.figure(figsize=(8, 6))
for i, idx in enumerate(gallery_idx):
    plt.subplot(rows, cols, i+1)
    img2d = X_test[idx,...,0]
    plt.imshow(img2d, cmap='gray_r' if USE_INVERT_GRAY else 'gray')
    plt.axis('off')
    plt.title(f"#{i}", fontsize=10)
plt.tight_layout(); plt.show()

In [ ]:
#@title 未知画像の予測
# 予測するカード番号
INDEXES = [0] #@param {type:"raw"}
if UNITS_COV5 > 0:
  TARGET_LAYER = 'conv5'
elif UNITS_COV4 > 0:
  TARGET_LAYER = 'conv4'
elif UNITS_COV3 > 0:
  TARGET_LAYER = 'conv3'
elif UNITS_COV2 > 0:
  TARGET_LAYER = 'conv2'
else:
  TARGET_LAYER = 'conv1'

plt.figure(figsize=(10, 3*len(INDEXES)))
for r, local_id in enumerate(INDEXES, start=1):
    idx = gallery_map[local_id]
    img4d = X_test[idx:idx+1]
    img2d = img4d[0, ..., 0]
    y_true = int(np.argmax(y_test[idx]))

    prob = model.predict(img4d, verbose=0)[0]
    pred = int(np.argmax(prob)); conf = float(np.max(prob))

    # Grad-CAM++
    heat, _ = gradcampp_for_layer(model, img4d, TARGET_LAYER)

    plt.subplot(len(INDEXES), 3, 3*r-2)
    plt.imshow(img2d, cmap='gray_r' if USE_INVERT_GRAY else 'gray'); plt.axis('off')
    plt.title(f"#{local_id} ")

    plt.subplot(len(INDEXES), 3, 3*r-1)
    plt.imshow(img2d, cmap='gray_r' if USE_INVERT_GRAY else 'gray')
    plt.imshow(heat, cmap='jet', alpha=0.5)
    plt.axis('off'); plt.title(f"pred={pred}, p={conf:.3f}")

plt.tight_layout(); plt.show()